In [43]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import InMemorySaver

load_dotenv()


True

In [26]:
llm = ChatGoogleGenerativeAI(
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0,
    model="gemini-2.5-flash",
)

In [27]:
# Define the state structure
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [ ]:
# function


def gen_joke(state: JokeState) -> str:
    prompt = f"You will be provided a topic. You task is to write a funniest joke on that topic. Topic: {state['topic']}"
    joke = llm.invoke(prompt).content
    return {"joke": joke}


def explain_joke(joke: str) -> str:
    prompt = f"You will be provided a joke. You task is to explain the joke in a simple way. Joke: {joke}"
    explanation = llm.invoke(prompt).content
    return {"explanation": explanation}

In [ ]:
# graph

graph = StateGraph(JokeState)

graph.add_node("gen_joke", gen_joke)
graph.add_node("explain_joke", explain_joke)

graph.add_edge(START, "gen_joke")
graph.add_edge("gen_joke", "explain_joke")
graph.add_edge("explain_joke", END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

In [37]:
config = {"configurable": {"thread_id": "1"}}
response = workflow.invoke({"topic": "pizza"}, config=config)

In [38]:
response

{'topic': 'pizza',
 'joke': 'Why did the pizza get a job?\n\nBecause it kneaded the dough!',
 'explanation': 'This joke is a **pun**! It plays on two phrases that sound exactly alike but have different meanings:\n\n1.  **"Kneaded the dough"**: This is what you do when you make pizza or bread. You work the dough with your hands.\n2.  **"Needed the dough"**: This means someone *needed money* (because "dough" is a slang term for money).\n\nThe humor comes from the idea that the pizza got a job because it literally "kneaded the dough" (as part of its nature), but the joke makes it sound like it "needed money" to live, just like a person would.'}

In [39]:
workflow.get_state(config)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it kneaded the dough!', 'explanation': 'This joke is a **pun**! It plays on two phrases that sound exactly alike but have different meanings:\n\n1.  **"Kneaded the dough"**: This is what you do when you make pizza or bread. You work the dough with your hands.\n2.  **"Needed the dough"**: This means someone *needed money* (because "dough" is a slang term for money).\n\nThe humor comes from the idea that the pizza got a job because it literally "kneaded the dough" (as part of its nature), but the joke makes it sound like it "needed money" to live, just like a person would.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09a3ff-766a-65b1-8006-4eab8565cefb'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-09-25T18:46:37.280196+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09a3f

In [41]:
list(workflow.get_state_history(config))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it kneaded the dough!', 'explanation': 'This joke is a **pun**! It plays on two phrases that sound exactly alike but have different meanings:\n\n1.  **"Kneaded the dough"**: This is what you do when you make pizza or bread. You work the dough with your hands.\n2.  **"Needed the dough"**: This means someone *needed money* (because "dough" is a slang term for money).\n\nThe humor comes from the idea that the pizza got a job because it literally "kneaded the dough" (as part of its nature), but the joke makes it sound like it "needed money" to live, just like a person would.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09a3ff-766a-65b1-8006-4eab8565cefb'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-09-25T18:46:37.280196+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09a3

## Time Travel

In [44]:
workflow.get_state(config={"configurable": {"thread_id": "1", "checkpoint": "1f09a3fe-c603-6e73-8004-f6ff7f8f9803"}})

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it kneaded the dough!', 'explanation': 'This joke is a **pun**! It plays on two phrases that sound exactly alike but have different meanings:\n\n1.  **"Kneaded the dough"**: This is what you do when you make pizza or bread. You work the dough with your hands.\n2.  **"Needed the dough"**: This means someone *needed money* (because "dough" is a slang term for money).\n\nThe humor comes from the idea that the pizza got a job because it literally "kneaded the dough" (as part of its nature), but the joke makes it sound like it "needed money" to live, just like a person would.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09a3ff-766a-65b1-8006-4eab8565cefb'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-09-25T18:46:37.280196+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09a3f

In [45]:
workflow.invoke(None, config={"configurable": {"thread_id": "1", "checkpoint": "1f09a3fe-c603-6e73-8004-f6ff7f8f9803"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza get a job?\n\nBecause it kneaded the dough!',
 'explanation': 'This joke is a **pun**! It plays on two phrases that sound exactly alike but have different meanings:\n\n1.  **"Kneaded the dough"**: This is what you do when you make pizza or bread. You work the dough with your hands.\n2.  **"Needed the dough"**: This means someone *needed money* (because "dough" is a slang term for money).\n\nThe humor comes from the idea that the pizza got a job because it literally "kneaded the dough" (as part of its nature), but the joke makes it sound like it "needed money" to live, just like a person would.'}

In [50]:
list(workflow.get_state_history(config))

[StateSnapshot(values={'topic': 'technology'}, next=('gen_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09a446-6986-61c8-8001-ffa88202883f'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-09-25T19:18:21.820154+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09a3fd-2125-6f25-8000-d88bd1693b99'}}, tasks=(PregelTask(id='c99b738e-7a07-d7ac-97bc-90f447c55510', name='gen_joke', path=('__pregel_pull', 'gen_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it kneaded the dough!', 'explanation': 'This joke is a **pun**! It plays on two phrases that sound exactly alike but have different meanings:\n\n1.  **"Kneaded the dough"**: This is what you do when you make pizza or bread. You work the dough with your hands.\n2.  **"Needed the dough"**: This means s

In [49]:
workflow.update_state(
    {"configurable": {"thread_id": "1", "checkpoint_id": "1f09a3fd-2125-6f25-8000-d88bd1693b99", "checkpoint_ns":""}},
    {"topic": "technology"}
)

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09a446-6986-61c8-8001-ffa88202883f'}}